## The Black-Scholes-Merton model

#### The stock dynamic 

$$
    dS_t = (r-q)S_tdt + S_t\sigma dW_t^Q
$$
$$
    S_T = S_t e^{(r-q-\frac{\sigma^2}{2})(T-t) +\sigma(W_T^Q - W_t^Q)}
$$
$W_t^Q$ is the Brownian Motion under the risk-neutral measure $Q$, $r$ is the risk-free rate, $q$ is the continuous dividend yield, $\sigma$ is the volatility, then the formula for the European Call and Put option:
$$
    C(t,T) = S_te^{-q(T-t)}\Phi(d1) -Ke^{-r(T-t)}\Phi(d2)
$$ 
$$
    P(t,T) =  Ke^{-r(T-t)}\Phi(-d2) - S_te^{-q(T-t)}\Phi(-d1)
$$
$$
    d1 = \frac{\log\left(\frac{S_t}{K} \right) + \left(r-q+\frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}}
$$
$$
    d2 = \frac{\log\left(\frac{S_t}{K} \right) + \left(r-q-\frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}}
$$

In [1]:
from functions import * 
import numpy as np

In [13]:
# Let's price a European call and put option with 6 months maturity
r = 0
sigma = .2
S0 = 3.76
K = 4 
q = 0
T = 1

C = Eurocall_Black_Scholes(S0, K, r, q, sigma, T)
P = Europut_Black_Scholes(S0, K, r, q, sigma, T)

print(f'Call price:\t{C:.4f}')
print(f'Put price:\t{P:.4f}')

Call price:	0.2037
Put price:	0.4437


### Black-Scholes-Merton model via Monte-Carlo simulations

The Monte-Carlo pricing method is based on the strong law of large nummbers (SLLN), namely it approximate the inegral
$$
    \Phi(S_t,t,T) = e^{-r(T-t)}E[\Phi(S_T,T)|\mathcal{F}_t] \approx e^{-r(T-t)} \frac{1}{n}\sum_{i=1}^n \Phi(S_T^{(i)},T)
$$
where are independent simulation of the undrlying process $S_T^{(i)}$. Thanks to the SLLN $\frac{1}{n}\sum_{i=1}^n \Phi(S_T^{(i)},T) \rightarrow E[\Phi(S_T,T)|\mathcal{F}_t]$ a.s. and $L^2$ as $n\rightarrow+\infty$.

### Simulation of the Geometric Brownian Motion (GBM)

In the case of the GBM we know the exact distribution of for the increments $r_t = \log\left(\frac{S_{t}}{S_{t-\Delta t}} \right)$, which are conditionally Gaussian with mean $(r-q-\frac{\sigma^2}{2})\Delta t$ and variance $\sigma^2\Delta t$ (basic properties of the Wiener Integral)

### Martingale property

A process $M_t$ is a martinagale w.r.t. to the filtration $\mathcal{F}_t$ if:

1) $E[M_t]<+\infty$
2) $M_t$ is $\mathcal{F}_t$ measurable
3) $E[M_t|\mathcal{F}_s] = M_s$ for $s<t$

The it holds the property:
$$
    E[M_t] = E[M_{t_0}] \quad \forall t>t_0 \quad E[M_t|\mathcal{F}_{t_0}] = M_{t_0} \quad \forall t>t_0
$$

In [8]:
# Let's simulate the GBM and chack if it is a martingale or not
np.random.seed(1)

r = 0
sigma = .2
S0 = 3.76
q = 0
T = .5
n_steps = 20

St = simulate_gbm(S0, r, q, sigma, T, n_steps, N=20)

In [9]:
# Check the martingale property

np.mean(St, axis=1)

array([3.76      , 3.76011064, 3.75995405, 3.7599961 , 3.76006852,
       3.75993549, 3.75994098, 3.75984688, 3.75979819, 3.75989768,
       3.75985061, 3.75996271, 3.76012059, 3.76024623, 3.76044154,
       3.76057118, 3.76064044, 3.76075901, 3.76085827, 3.76081657,
       3.7606451 ])

In [17]:
# Let's price a European call and put option with 6 months maturity
np.random.seed(1)

r = 0
sigma = .2
S0 = 3.76
K = 4 
q = 0
T = 1

n_steps = int(252*T)
N = 20

St = simulate_gbm(S0, r, q, sigma, T, n_steps, N)

# Since we have the exact conditional distribution we can simulate directly with one step to price the European style options
C = EuropeanOption(St[-1], K, r, T, 1)
P = EuropeanOption(St[-1], K, r, T, -1)

print(f'Mc Call price:\t{C:.4f}')
print(f'MC Put price:\t{P:.4f}')

Mc Call price:	0.2030
MC Put price:	0.4438
